<a href="https://colab.research.google.com/github/Samurarahman/CSE438/blob/main/CSE438_A01_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Mount Google Drive
We mount Google Drive to access our input video (`12s.mp4`) and save the output directly to the same folder.


In [ ]:
#  Mount Google Drive to store output
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Install and Import Libraries
We install `opencv-python-headless` and import all necessary Python libraries, including OpenCV, NumPy, and Colab utilities like `drive` and `Video`.


In [ ]:
!pip install opencv-python-headless --quiet

import cv2
import numpy as np
import os
import shutil
from matplotlib import pyplot as plt
from IPython.display import Video, display


### Define Video Paths
We set the file paths for the input and output videos using the Drive directory. The output will be saved in the same folder as the input video.


In [ ]:
#  Input video from Drive
INPUT_PATH = '/content/drive/MyDrive/cse438/assignment 1/12s.mp4'
OUTPUT_PATH = '/content/drive/MyDrive/cse438/assignment 1/output_hist_eq_grid.avi'


###  Initialize Video Capture and Writer
We use `cv2.VideoCapture` to read the input video and `cv2.VideoWriter` to write a new video file. The output frame is a 2×2 grid, so its resolution is doubled in both width and height.


In [ ]:
#  Open video
cap = cv2.VideoCapture(INPUT_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
width, height = int(cap.get(3)), int(cap.get(4))

#  Output writer: double width and height for 2x2 grid
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'XVID'), fps,
                      (width * 2, height * 2))


###  Create RGB Histogram Panel
This function calculates and draws RGB histograms from any input frame. It uses `cv2.calcHist` to extract histograms for each color channel and draws them using OpenCV lines.


In [ ]:
# RGB histogram panel generator
def make_histogram_panel(bgr_frame, size=(320, 240), bins=64):
    panel = np.zeros((size[1], size[0], 3), dtype=np.uint8)
    colors = ['b', 'g', 'r']
    for i, col in enumerate(colors):
        hist = cv2.calcHist([bgr_frame], [i], None, [bins], [0, 256])
        hist = cv2.normalize(hist, hist, 0, size[1], cv2.NORM_MINMAX).flatten()
        bin_width = int(size[0] / bins)
        for x in range(1, bins):
            cv2.line(panel,
                     (bin_width*(x-1), size[1] - int(hist[x-1])),
                     (bin_width*x, size[1] - int(hist[x])),
                     (255 if col == 'b' else 0,
                      255 if col == 'g' else 0,
                      255 if col == 'r' else 0),
                     2)
    return panel


###  Process Video Frame-by-Frame
For each frame:
- We copy the original image.
- Convert it to YCrCb color space.
- Apply histogram equalisation to the Y (luminance) channel only.
- Convert back to BGR.
- Create RGB histograms for both original and equalised frames.
- Combine all four components into a 2x2 grid.
- Write the grid to the output video.


In [ ]:
# Process video
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original = frame.copy()

    # Equalise luminance in YCrCb
    ycrcb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    equalised = cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

    # Histograms
    orig_hist = make_histogram_panel(original, size=(width, height))
    eq_hist = make_histogram_panel(equalised, size=(width, height))

    # Compose 2x2 grid
    top = np.hstack((original, equalised))
    bottom = np.hstack((orig_hist, eq_hist))
    grid = np.vstack((top, bottom))

    out.write(grid)
    frame_count += 1

cap.release()
out.release()
print(f"✅ Done. {frame_count} frames processed and saved to {OUTPUT_PATH}")


✅ Done. 320 frames processed and saved to /content/drive/MyDrive/cse438/assignment 1/output_hist_eq_grid.avi


### Convert to MP4 and Display Output
To ensure smooth playback inside Colab, we convert the `.avi` file to `.mp4` using FFmpeg, then embed it using IPython’s Video player.


In [ ]:
# Optional: convert to .mp4 for playback in Colab
OUTPUT_MP4 = '/content/drive/MyDrive/cse438/assignment 1/output_hist_eq_grid.mp4'
!ffmpeg -i "{OUTPUT_PATH}" -vcodec libx264 -crf 24 "{OUTPUT_MP4}" -y -loglevel quiet

# Preview in notebook
Video(OUTPUT_MP4, embed=True, width=720)


### Input Video from Pixabay

We use the following free stock video from Pixabay:

**URL**: https://pixabay.com/videos/woman-model-dancing-silhouette-vj-185787/

**License**: Pixabay Content License — royalty-free for personal and commercial use, no attribution required.



**Drive link**:https://drive.google.com/drive/folders/1YwsiINqqWcjogFZVcTGpO44oe0JPQkEx?usp=sharing